In [2]:
from functools import wraps

def decorator(ar, argument="burp"):
    print("decorator")
    def real_decorator(function):
        print("real decorator")
        @wraps(function)
        def wrapper(*args, **kwargs):
            print(argument)
            retval = function(*args, **kwargs)
            return retval
        return wrapper
    return real_decorator


@decorator("ok")
def ops(ciao, pao, miao, tao):
    print("ops", ciao)

decorator
real decorator


In [3]:
ops("ok", "ok", "ok", "ok")

burp
ops ok


In [5]:
import time

def counter(n):
    try:
        counter = 0
        while counter <= n:
            counter += 1
            yield counter
            print("ciao")
    finally:
        print(f"Nice! You counted to {counter}")


for count in counter(6):
    print(f"count={count}")
    time.sleep(1)

count=1
ciao
count=2
ciao
count=3
ciao
count=4
ciao
count=5
ciao
count=6
ciao
count=7
ciao
Nice! You counted to 7


In [75]:
class Test:
    def __init__(self):
        self.nodes = {}
        pass
    
    def __setattr__(self, name, value):
        print(name, value)
        super().__setattr__(name, value)
    
    def test(self, node, value):
        self.nodes[node] = value

def ciao():
    print("ciao")

In [80]:
lista = [ciao]

In [81]:
lista

[<function __main__.ciao()>]

In [82]:
lista.remove(ciao)

In [83]:
lista

[]

In [92]:
from __future__ import annotations

import re
import asyncio
from typing import Any, Callable, Union

class Observable(object):
    def __init__(self):
        self._callbacks = []

    def subscribe(self, callback: Callable):
        self._callbacks.append(callback)
    
    def unsubscribe(self, callback: Callable):
        if callback in self._callbacks: 
            self._callbacks.remove(callback)

    def _call_subscribers(self, *attrs, **kwargs):
        fn_return = []
        
        for fn in self._callbacks:
            if asyncio.iscoroutinefunction(fn):
                fn_return.append(asyncio.create_task(fn(*attrs, **kwargs)))
            else:
                fn_return.append(fn(*attrs, **kwargs))

TopicDictCallbackType = Callable[[str, Any], bool]


class TopicNode(dict):
    def __init__(self, value: Any = None, sub_topic: dict[str, TopicNode] = {}):
        super().__init__(sub_topic)
        self._value = value

    def __str__(self):
        topic_child_str = ", ".join(
            "%s: %s" % (key, self.get(key, return_value=False).__str__())
            for key in super().keys()
        )
        return "(%s, {%s})" % (self._value, topic_child_str)

    def __repr__(self):
        return self.__str__()

    def dict_value(self):
        return {k: v.value for k, v in self.items()}

    @property
    def value(self):
        return self._value

    @value.setter
    def value(self, value):
        self._value = value


# TODO: swap super(TopicNode, self) with TopicNode.method(self, )
class TopicDict(TopicNode, Observable):
    def __init__(self, include_topics: list[str] = [], exclude_topics: list[str] = []):
        TopicNode.__init__(self)
        Observable.__init__(self)
        self._include_topics = list()
        self._exclude_topics = list()
        self.add_include_topic(*include_topics)
        self.add_exclude_topic(*exclude_topics)

    @staticmethod
    def _topic_to_lst(topic_path: str) -> list:
        return topic_path.strip("/").split("/")

    @staticmethod
    def topic_get_head(topic: str) -> Union[tuple[str, str], bool]:
        topic = topic.strip("/")
        last_slash_index = topic.rfind("/")

        if last_slash_index == -1:
            return False

        return topic[last_slash_index + 1 :], topic

    def add_include_topic(self, *regex_patterns: list[str]):
        for regex_pattern in regex_patterns:
            self._include_topics.append(re.compile(regex_pattern))

    def add_exclude_topic(self, *regex_patterns: list[str]):
        for regex_pattern in regex_patterns:
            self._exclude_topics.append(re.compile(regex_pattern))

    def _get_parent_by_topic(self, topic_path: str):

        topic_path_list = self._topic_to_lst(topic_path)
        return self.get(topic_path_list[:-1], return_value=False), topic_path_list[-1]

    def get(
        self,
        topic_path: Union[str, list],
        default: Any = None,
        return_value: bool = True,
    ) -> Union[Any, TopicDict]:

        if not isinstance(topic_path, list):
            topic_path = self._topic_to_lst(topic_path)

        topic_node = self

        for topic_lvl in topic_path:

            if topic_lvl not in topic_node:
                return default

            topic_node = super(TopicDict, topic_node).__getitem__(topic_lvl)

        return topic_node.value if return_value else topic_node

    def get_obj(
        self, topic_path: Union[str, list], default: TopicNode = TopicNode(None)
    ):
        return self.get(topic_path, default, return_value=False)

    def set(self, topic_path: str, value: Any, force: bool = False):

        if not force:
            if self._include_topics and not any(
                regex_include.search(topic_path)
                for regex_include in self._include_topics
            ):
                return False

            if self._exclude_topics and any(
                regex_exclude.search(topic_path)
                for regex_exclude in self._exclude_topics
            ):
                return False

        topic_node = self

        for topic_lvl in self._topic_to_lst(topic_path):
            topic_node = topic_node.setdefault(topic_lvl, TopicDict())

        if isinstance(value, TopicDict):
            topic_parent_node, topic_label = self._get_parent_by_topic(topic_path)
            super(TopicDict, topic_parent_node).__setitem__(topic_label, value)

        else:
            topic_node._value = value

        Observable._call_subscribers(self, topic_path, value)

    def _del(self, topic_path: str):

        topic_parent_node, topic_label = self._get_parent_by_topic(topic_path)

        if not topic_parent_node:
            return False

        return topic_parent_node.pop(topic_label, False)

    def __getitem__(self, topic_path) -> Any:
        return self.get(topic_path)

    def __setitem__(self, topic_path, value):
        self.set(topic_path, value)

    def __delitem__(self, topic_path):
        self._del(topic_path)

    @property
    def value(self):
        return self._value

    @value.setter
    def value(self, value):
        self._value = value

        Observable._call_subscribers(self, "", value)

In [93]:
ciao = TopicDict()

In [96]:
ciao.subscribe(lambda topic, value: print('listener', topic, value) )

In [97]:
async def async_listner(topic, value):
    print('async listner', topic, value)
    
ciao.subscribe(async_listner)

In [98]:
ciao.add_include_topic(r"^included/path", r"/included2/second/path")
ciao.set('included/path/end', 'included/path/end')
ciao['/included2/second/path'] = '/included2/second/path'
ciao['/included2/second/path2'] = '/included2/second/path2'
ciao.set('root/path', 'root/path')
ciao.set('root/xpath', 'root/xpath')
ciao.set('root/xpath/ciao', 'root/xpath/ciao')

listener included/path/end included/path/end
listener /included2/second/path /included2/second/path
listener /included2/second/path2 /included2/second/path2


False

async listner included/path/end included/path/end
async listner /included2/second/path /included2/second/path
async listner /included2/second/path2 /included2/second/path2


In [99]:
ciao.unsubscribe(async_listner)

In [100]:
ciao.set('included/path/end', 'included/path/end')
ciao['/included2/second/path'] = '/included2/second/path'
ciao['/included2/second/path2'] = '/included2/second/path2'
ciao.set('root/path', 'root/path')
ciao.set('root/xpath', 'root/xpath')
ciao.set('root/xpath/ciao', 'root/xpath/ciao')

listener included/path/end included/path/end
listener /included2/second/path /included2/second/path
listener /included2/second/path2 /included2/second/path2


False

In [101]:
ciao.unsubscribe(async_listner)

In [2]:
from notebook.auth import passwd
passwd('','sha1')

'sha1:7e1b6aa39294:7cf4db9c130ca551c0cf55bf0bc7ea29a55f8c0a'

In [1]:
class PrintObserver():

    def on_next(self, value):
        print("Received {0}".format(value))

    def on_completed(self):
        print("Done!")

    def on_error(self, error):
        print("Error Occurred: {0}".format(error))

In [3]:
PrintObserver().on_completed()

Done!


In [4]:
PrintObserver().on_next(10)

Received 10


In [5]:
ciao = PrintObserver()


In [6]:
ciao.on_next(10)

Received 10


In [30]:
!pip uninstall rx

Found existing installation: Rx 3.2.0
Uninstalling Rx-3.2.0:
  Would remove:
    /home/muttley/.virtualenvs/playground/lib/python3.10/site-packages/Rx-3.2.0.dist-info/*
    /home/muttley/.virtualenvs/playground/lib/python3.10/site-packages/rx/*
Proceed (y/n)? ^C
ERROR: Operation cancelled by user


In [4]:
from rx import create

In [5]:
def ciao(observer, scheduler):

    observer.on_next("Alpha")

    def on_next(a):
        observer.on_next(a)

    def on_completed():
        observer.on_completed()

    ciao.go_on = on_next
    ciao.finish = on_completed

In [6]:
source = create(ciao)

source.subscribe(
    on_next = lambda i: print("Received {0}".format(i)),
    on_error = lambda e: print("Error Occurred: {0}".format(e)),
    on_completed = lambda: print("Done!"),
)

Received Alpha


In [25]:
ciao.go_on("test")
ciao.finish()
ciao.go_on("test")

Received test
Done!


In [2]:
def push_five_strings(observer, scheduler):
    observer.on_next("Alpha")
    observer.on_next("Beta")
    observer.on_next("Gamma")
    observer.on_next("Delta")
    observer.on_next("Epsilon")
    observer.on_completed()

source = create(push_five_strings)

source.subscribe(
    on_next = lambda i: print("Received {0}".format(i)),
    on_error = lambda e: print("Error Occurred: {0}".format(e)),
    on_completed = lambda: print("Done!"),
)

Received Alpha
Received Beta
Received Gamma
Received Delta
Received Epsilon
Done!


In [3]:
from rx import of

source = of("Alpha", "Beta", "Gamma", "Delta", "Epsilon")

source.subscribe(lambda value: print("Received {0}".format(value)))

Received Alpha
Received Beta
Received Gamma
Received Delta
Received Epsilon


In [27]:
import rx

def lowercase():
    def _lowercase(source):
        def subscribe(observer, scheduler = None):


            def on_next(value):
                observer.on_next("1")
                observer.on_next("ciao")

            return source.subscribe(
                on_next,
                observer.on_error,
                observer.on_completed,
                scheduler)
        return rx.create(subscribe)
    return _lowercase

rx.of("Alpha", "Beta", "Gamma", "Delta", "Epsilon").pipe(
        lowercase()
     ).subscribe(lambda value: print("Received {0}".format(value)))

Received 1
Received ciao
Received 1
Received ciao
Received 1
Received ciao
Received 1
Received ciao
Received 1
Received ciao


In [1]:
import multiprocessing
import random
import time
from threading import current_thread

import rx
from rx.scheduler import ThreadPoolScheduler
from rx import operators as ops


def intense_calculation(value):
    # sleep for a random short duration between 0.5 to 2.0 seconds to simulate a long-running calculation
    time.sleep(random.randint(5, 20) * 0.1)
    print("intense calculation: {0}".format(current_thread().name))
    return value


# calculate number of CPUs, then create a ThreadPoolScheduler with that number of threads
optimal_thread_count = multiprocessing.cpu_count()
pool_scheduler = ThreadPoolScheduler(optimal_thread_count)

# Create Process 1
rx.of("Alpha", "Beta", "Gamma", "Delta", "Epsilon").pipe(
    ops.map(lambda s: intense_calculation(s)),
    ops.subscribe_on(pool_scheduler)
).subscribe(
    on_next=lambda s: print("PROCESS 1: {0} {1}".format(current_thread().name, s)),
    on_error=lambda e: print(e),
    on_completed=lambda: print("PROCESS 1 done!"),
)

print("1")

# Create Process 2
rx.range(1, 10).pipe(
    ops.map(lambda s: intense_calculation(s)),
    ops.subscribe_on(pool_scheduler)
).subscribe(
    on_next=lambda i: print("PROCESS 2: {0} {1}".format(current_thread().name, i)),
    on_error=lambda e: print(e),
    on_completed=lambda: print("PROCESS 2 done!"),
)

print("2")

# Create Process 3, which is infinite
rx.interval(1).pipe(
    ops.map(lambda i: i * 100),
    ops.map(lambda s: intense_calculation(s)),
    ops.observe_on(pool_scheduler),
).subscribe(
    on_next=lambda i: print("PROCESS 3: {0} {1}".format(current_thread().name, i)),
    on_error=lambda e: print(e),
)

print("3")
input("Press Enter key to exit\n")

1
2
3
intense calculation: ThreadPoolExecutor-0_0
PROCESS 1: ThreadPoolExecutor-0_0 Alpha
intense calculation: ThreadPoolExecutor-0_1
PROCESS 2: ThreadPoolExecutor-0_1 1
intense calculation: ThreadPoolExecutor-0_1
PROCESS 2: ThreadPoolExecutor-0_1 2
intense calculation: Thread-8
PROCESS 3: ThreadPoolExecutor-0_2 0
intense calculation: ThreadPoolExecutor-0_0
PROCESS 1: ThreadPoolExecutor-0_0 Beta
intense calculation: Thread-9
PROCESS 3: ThreadPoolExecutor-0_2 100
intense calculation: ThreadPoolExecutor-0_1
PROCESS 2: ThreadPoolExecutor-0_1 3
intense calculation: ThreadPoolExecutor-0_0
PROCESS 1: ThreadPoolExecutor-0_0 Gamma


''

In [23]:
import rx
from rx import operators as ops

def customer_for_id(customer_id):
    stmt = f"SELECT * FROM CUSTOMER WHERE CUSTOMER_ID = {id}"
    return rx.of(f"Alpha {customer_id}")


# Query customers with IDs 1, 3, and 5
a = rx.from_([1, 3, 5])
b = rx.from_([1, 3, 5])

rx.zip(a,b).subscribe(
    on_next=lambda i: print("PROCESS 2: {1}".format(i)),
    on_error=lambda e: print(e),
    on_completed=lambda: print("PROCESS 2 done!"),
)

input("Press Enter key to exit\n")

PROCESS 2 done!


'd'

In [22]:
import rx
from rx import operators as op
from rx.subject import Subject

root = Subject()

foo = root.pipe(
    op.map( lambda x : x + 1 ),
    op.do_action(lambda r: print("foo(x) = %s (expensive)" % str(r))),
    op.share()
)

bar_foo = foo.pipe(
    op.map( lambda x : x * 2 ),
    op.do_action(lambda r: print("bar(foo(x)) = %s" % str(r)))
)

bar_foo.pipe(
    op.zip(foo),
    op.map(lambda i: i[0]+i[1]),
    op.do_action(lambda r: print("foo(x) + bar(foo(x)) = %s" % str(r)))
).subscribe()


print("-------------")
root.on_next(10)
print("-------------")

-------------
foo(x) = 11 (expensive)
bar(foo(x)) = 22
foo(x) + bar(foo(x)) = 33
-------------


In [21]:
xs = rx.from_marbles("1-2-3")
ys = rx.from_marbles("1-2-3")
rx.merge(xs,ys).pipe(ops.to_list()).run()

KeyboardInterrupt: 

In [26]:

from rx import operators as op

test = rx.of(1,2,3,4,5,6)
test1 = rx.of(4,8,12,16,20)
test2 = rx.of(5,10,15,20,25)
sub1 = test.pipe(
    op.zip(test1, test2)
)
sub1.subscribe(lambda x: print("The value is {0}".format(x)))

In [27]:
import rx
from rx import operators as ops
import operator

a = rx.of(1, 2, 3, 4)
b = rx.of(2, 2, 4, 4)

a.pipe(
    ops.zip(b), # returns a tuple with the items of a and b
    ops.map(lambda z: operator.mul(z[0], z[1]))
).subscribe(print)

In [39]:
import rx
from rx import operators as ops
import operator

a = rx.of(1, 2, 3, 4)
b = rx.of(2, 2, 4, 4)

a.pipe(
    ops.zip(b),
    ops.starmap(operator.mul)
).subscribe(print)

2
4
12
16


In [40]:
class MyObserver(rx.core.Observer):
    def on_next(self, x):
        print("Got: %s" % x)

    def on_error(self, e):
        print("Got error: %s" % e)

    def on_completed(self):
        print("Sequence completed")

xs = rx.from_(range(10))
d = xs.subscribe(MyObserver())

Got: 0
Got: 1
Got: 2
Got: 3
Got: 4
Got: 5
Got: 6
Got: 7
Got: 8
Got: 9
Sequence completed


In [56]:
xs = rx.range(1, 5)
ys = rx.from_("abcde")
zs = rx.merge(xs,ys).subscribe(print)

1
a
b
c
d
e
2
3
4


In [57]:
xs.pipe(ops.merge(ys)).subscribe(print)

1
a
b
c
d
e
2
3
4


In [69]:
marbles = rx.from_marbles('a--b--c-|', lookup={'a': 1, 'b': 2, 'c': 3}).pipe(ops.to_marbles())

marbles.run()

'1---2---3--|'

In [102]:
from rx.subject import ReplaySubject, Subject

#stream = Subject()
stream = ReplaySubject()
stream.on_next(41)

#d = stream.subscribe(lambda x: print("Got: %s" % x))

stream.on_next(42)
stream.on_next(42)

#marbles = rx.from_marbles('a--b--c-|', lookup={'a': 1, 'b': 2, 'c': 3}).pipe(ops.to_marbles())
marbles = rx.from_([1,2,3,4,5])

rx.merge(stream, marbles).subscribe(lambda x: print("merge %s" % x))

#d.dispose()
stream.on_next(43)


merge 41
merge 1
merge 2
merge 3
merge 4
merge 5
merge 42
merge 42
merge 43


In [101]:
stream.on_next(43)

merge 43


In [72]:
!pip install nest_asyncio

You should consider upgrading via the '/home/muttley/Projects/SecondLevelDegrees/courses/Thesis/source/venv/bin/python -m pip install --upgrade pip' command.


In [73]:
import nest_asyncio
nest_asyncio.apply()

In [76]:
import asyncio
import rx
import rx.operators as op
import time


source1: rx.Observable = rx.from_marbles('a-b-3-4-|', )

async def main(loop):
    x = source1.subscribe(
        on_next = lambda i: print("1: {0}".format(i)),
        on_error = lambda e: print("Error Occurred: {0}".format(e)),
        on_completed = lambda: print("1 Done!"),p
    )
    await source1


loop = asyncio.get_event_loop()
loop.run_until_complete(main(loop))
loop.close()
print('end')

1: a
1: b
1: 3
1: 4
1 Done!


RuntimeError: Cannot close a running event loop

In [109]:
import asyncio
from random import randint

async def periodic():
    while True:
        delay = randint(1, 6)
        await asyncio.sleep(delay)
        print('periodic', delay)


def stop():
    task.cancel()

loop = asyncio.get_event_loop()
loop.call_later(50, stop)
task = loop.create_task(periodic())

try:
    loop.run_until_complete(task)
except asyncio.CancelledError:
    pass

periodic 1
periodic 3
periodic 6
periodic 2


KeyboardInterrupt: 

In [128]:
async def delay_callback(fn):
    delay = randint(1, 6)
    await asyncio.sleep(delay)
    print('delay_callback', delay)
    fn()

In [10]:
import time
from random import randint
starttime = time.time()
while True:
    print("tick")
    time.sleep(randint(1, 6))

tick
tick
tick
tick
tick
tick


KeyboardInterrupt: 

In [12]:
import rx
import rx.operators as op
import time
from random import randint

import multiprocessing
from rx.scheduler import ThreadPoolScheduler
optimal_thread_count = multiprocessing.cpu_count()
pool_scheduler = ThreadPoolScheduler(optimal_thread_count)


def ciao(qualcosa):
    def _ciao(observer, scheduler):
        # def on_next(a):
        #     observer.on_next(a)
        #
        # def on_completed():
        #     observer.on_completed()
        #
        # ciao.go_on = on_next
        # ciao.finish = on_completed

        observer.on_next(qualcosa)

        while True:
            delay = randint(1, 6)
            time.sleep(delay)
            observer.on_next(delay)

    return _ciao

def lowercase():
    memory = 0
    def _lowercase(source):
        def subscribe(observer, scheduler = None):
            def on_next(value):
                observer.on_next(value.lower())
                observer.on_next(value + ' e due')

                # nonlocal memory
                # if memory >= 3:
                #     observer.on_next(value.lower())
                #     memory = 0
                # else:
                #     memory+=1


            return source.subscribe(
                on_next,
                observer.on_error,
                observer.on_completed,
                scheduler)
        return rx.create(subscribe)
    return _lowercase

ok = ciao("CIAO")

source = rx.create(ok).pipe(op.subscribe_on(pool_scheduler), op.share())

forkA = source.pipe(op.do_action(lambda x: print(f"forkA {x}")))

#forkA.subscribe(print)

forkB = source.pipe(
    op.map(lambda x: f"{x} + map"),
    lowercase(),
    #op.do_action(lambda x: print(f"forkB {x}"))
).subscribe(print, scheduler=pool_scheduler)

# forkA.pipe(op.merge(forkB)).subscribe(
#     on_next = lambda i: print("\nReceived {0}\n".format(i)),
#     on_error = lambda e: print("Error Occurred: {0}".format(e)),
#     on_completed = lambda: print("Done!")
# )

print('end')

#source.connect()

print('end')

time.sleep(10)

endciao + map
end

CIAO + map e due
5 + map
5 + map e due
4 + map
4 + map e due
2 + map
2 + map e due
5 + map
5 + map e due


In [58]:
source = of("Alpha", "Beta", "Gamma", "Delta", "Epsilon")

source.subscribe(lambda value: print("Received {0}".format(value)), scheduler=pool_scheduler)

NameError: name 'of' is not defined

In [1]:
import rx
import rx.operators as op
import time
from random import randint
import asyncio

import nest_asyncio
nest_asyncio.apply()

In [2]:

def lowercase():
    memory = 0
    def _lowercase(source):
        def subscribe(observer, scheduler = None):
            def on_next(value):
                observer.on_next(value.lower())

            return source.subscribe(
                on_next,
                observer.on_error,
                observer.on_completed,
                scheduler)
        return rx.create(subscribe)
    return _lowercase

def uppercase(value):
    return value.upper()

def append(value):
    def _append():
        return value + " ciao"
    return rx.from_callable(lambda: _append())

source = rx.interval(1).pipe(op.publish())

forkA = source.pipe(
    op.do_action(lambda x: print(f"forkA {x}"))
)

#forkA.subscribe(print)

forkB = source.pipe(
    op.map(lambda x: f"{x} + MAP"),
    op.do_action(lambda x: print(f"forkB {x}")),
    lowercase(),
    op.do_action(lambda x: print(f"forkB lower {x}")),
    op.map(lambda x: uppercase(x)),
    op.do_action(lambda x: print(f"forkB upper {x}")),
    op.flat_map(lambda x: append(x)),
    op.do_action(lambda x: print(f"forkB append {x}")),
)

#forkB.subscribe(print)

forkA.pipe(op.merge(forkB)).subscribe(
    on_next = lambda i: print("\nReceived: {0}\n".format(i)),
    on_error = lambda e: print("Error Occurred: {0}".format(e)),
    on_completed = lambda: print("Done!")
)

source.connect()

print('end')

input("Press Enter key to exit\n")

end
forkA 0

Received: 0

forkB 0 + MAP
forkB lower 0 + map
forkB upper 0 + MAP
forkB append 0 + MAPciao

Received: 0 + MAPciao

forkA 1

Received: 1

forkB 1 + MAP
forkB lower 1 + map
forkB upper 1 + MAP
forkB append 1 + MAPciao

Received: 1 + MAPciao



''

In [9]:
from asyncio import Future

def test():
    future = Future()
    future.set_result(42)
    return future

rx.start_async(test()).subscribe(print)

TypeError: 'Future' object is not callable

In [2]:
import asyncio
import functools
import rx
from rx.scheduler.eventloop import AsyncIOScheduler
from rx.disposable import Disposable

import nest_asyncio
nest_asyncio.apply()

loop = asyncio.get_event_loop()
scheduler = AsyncIOScheduler(loop)

def from_aiter(loop):
    def on_subscribe(observer, scheduler):
        async def _aio_sub():
            try:
                await asyncio.sleep(1)
                observer.on_next(1)
                await asyncio.sleep(1)
                observer.on_next(2)

                # loop.call_soon(
                #     observer.on_completed)
            except Exception as e:
                loop.call_soon(
                    functools.partial(observer.on_error, e))

        task = loop.create_task(_aio_sub())
        return Disposable(lambda: task.cancel())

    return rx.create(on_subscribe)

async def main(loop, scheduler):

    source1: rx.Observable = from_aiter(loop)

    source1.subscribe(
        on_next = lambda i: print("1: {0}".format(i)),
        on_error = lambda e: print("Error Occurred: {0}".format(e)),
        on_completed = lambda: print("1 Done!"),
        scheduler=scheduler
    )
    print("after source")


loop.create_task(main(loop, scheduler))
loop.run_forever()
print('end')

after source
1: 1
1: 2


KeyboardInterrupt: 

In [1]:
import asyncio
import rx
import rx.operators as ops
from rx.scheduler.eventloop import AsyncIOScheduler

import nest_asyncio
nest_asyncio.apply()

import threading
import time
start = time.time()


def log(value: str):
    print(f"{time.time() - start:.3f} {current_thread().name} {value}")

async def foo():
    await asyncio.sleep(random.randint(1, 6))
    return 42


def intervalRead(rate, fun) -> rx.Observable:
    loop = asyncio.get_event_loop()
    return rx.interval(rate).pipe(
        ops.map(lambda i: rx.from_future(loop.create_task(fun()))),
        ops.merge_all(),
        # ops.debounce(3.0)
    )

async def main(loop):

    obs = intervalRead(2, foo)
    obs.subscribe(
        on_next=lambda item: log(item),
        # scheduler=AsyncIOScheduler(loop)
    )

loop = asyncio.get_event_loop()
loop.create_task(main(loop))
loop.run_forever()

KeyboardInterrupt: 

In [1]:
import asyncio
import rx
import rx.operators as ops
from rx.scheduler.eventloop import AsyncIOScheduler

import nest_asyncio
nest_asyncio.apply()

async def test(a, b):
    await asyncio.sleep(1)
    print("inside")
    await asyncio.sleep(1)
    return 42

async def main(loop):

    rx.from_future(
        loop.create_task(test(1,2))
    ).subscribe(
        on_next=lambda item: print(item),
        scheduler=AsyncIOScheduler(loop)
    )

loop = asyncio.get_event_loop()
loop.create_task(main(loop))
print("ciao")
loop.run_forever()

ciao
inside
42


KeyboardInterrupt: 

In [1]:
import asyncio
import functools
import rx
from rx.scheduler.eventloop import AsyncIOScheduler
from rx.disposable import Disposable

import nest_asyncio
nest_asyncio.apply()

def from_aiter(iter, loop):
    def on_subscribe(observer, scheduler):
        async def _aio_sub():
            try:
                async for i in iter:
                    observer.on_next(i)
                loop.call_soon(
                    observer.on_completed)
            except Exception as e:
                loop.call_soon(
                    functools.partial(observer.on_error, e))

        task = asyncio.ensure_future(_aio_sub(), loop=loop)
        return Disposable(lambda: task.cancel())

    return rx.create(on_subscribe)


async def ticker(delay, to):
    """Yield numbers from 0 to `to` every `delay` seconds."""
    for i in range(to):
        yield i
        await asyncio.sleep(delay)


async def main(loop):
    done = asyncio.Future()

    def on_completed():
        print("completed")
        done.set_result(0)

    disposable = from_aiter(ticker(2, 10), loop).subscribe(
        on_next=lambda i: print("next: {}".format(i)),
        on_error=lambda e: print("error: {}".format(e)),
        on_completed=on_completed,
    )

    await done
    disposable.dispose()

loop = asyncio.get_event_loop()
loop.run_until_complete(main(loop))

next: 0
next: 1
next: 2
next: 3
next: 4
next: 5
next: 6
next: 7
next: 8
next: 9
completed


In [2]:
# subscribe(..., scheduler=) / interval(..., scheduler=), ops.observe_on(scheduler)

import asyncio
import rx
import rx.operators as ops
from rx.scheduler.eventloop import AsyncIOScheduler

from threading import current_thread

import multiprocessing
from rx.scheduler import ThreadPoolScheduler
optimal_thread_count = multiprocessing.cpu_count()
pool_scheduler = ThreadPoolScheduler(optimal_thread_count)

import nest_asyncio
nest_asyncio.apply()

loop = asyncio.get_event_loop()
_scheduler = AsyncIOScheduler(loop)

async def foo():
    print("foo: {}".format(current_thread().name))
    await asyncio.sleep(1)
    return 42


def intervalRead(rate, fun) -> rx.Observable:
    print("intervalRead: {}".format(current_thread().name))

    #loop = asyncio.get_event_loop()

    return rx.interval(rate,
                       # Alternative
                       # do the same of "scheduler=" param in obs.subscribe()
                       #scheduler=_scheduler
    ).pipe(
        ops.do_action(lambda _: print("do_action1: {}".format(current_thread().name))),
        ops.map(lambda i: rx.from_future(loop.create_task(fun()))),
        # switch following operators on another thread
        ops.observe_on(pool_scheduler),
        ops.do_action(lambda _: print("do_action2: {}".format(current_thread().name))),
        ops.merge_all()
    )


async def main(loop):
    obs = intervalRead(1, foo)
    obs.subscribe(
        on_next=lambda item: print("subscribe: {} ({})".format(current_thread().name, item)),
        # set scheduler for interval (ie. observable/create/on_subscribe fn)
        scheduler=AsyncIOScheduler(loop)
    )

    print('end')


loop.create_task(main(loop))
loop.run_forever()

intervalRead: MainThread


KeyboardInterrupt: 

In [1]:
# Add Item to Subject and wait the end of computation (by future.set_result())
# the order is not preserved on parallelized/asyncio.gather() (ie. wait the end of computation, before start the new one)
# SOLUTION WITH QUEUE IN THE NEXT CEL!!

import asyncio
from asyncio.futures import Future
import time
from typing import NamedTuple
import rx.operators as ops
from rx.subject import Subject
from rx.scheduler.eventloop import AsyncIOScheduler

import nest_asyncio
nest_asyncio.apply()

start = time.time()
def ts():
    return f"{time.time() - start:.3f}"

ACTION_DURATION = 2.0
Data = NamedTuple('Data', [('id', int), ('future', Future[int])])
subject = Subject()

async def async_action(id: int = 0):
    """ Some async processing, like sending/writing data. """
    print(f"{ts()} Async action  started {id}")
    await asyncio.sleep(ACTION_DURATION)
    print(f"{ts()} Async action finished {id}")
    return id


async def setup():
    """ Setting up the Rx subject to make sure all work is done
        in sequence without overlaps.
    """

    def sequencer(data: Data):
        task = asyncio.create_task(async_action(data.id))
        task.add_done_callback(
            lambda f: data.future.set_result(f.result())
        )
        return task

    subject.pipe(
        ops.do_action(lambda f: sequencer(f))
    ).subscribe(
        scheduler=AsyncIOScheduler(asyncio.get_event_loop())
    )


async def add(id: int):
    """ Add "some work" and await until it is done. """
    future = asyncio.Future()
    subject.on_next(Data(id, future))
    return await future


async def main():
    await setup()

    # add work in any fashion (code I cannot influence)
    print(f"{ts()} before 1")
    await add(1)
    print(f"{ts()} after  1")

    await asyncio.sleep(0.1)

    print(f"{ts()} before 2")
    await add(2)
    print(f"{ts()} after  2")

    print(f"{ts()} before 4, 5, 6")
    await asyncio.gather(add(4), add(5), add(6))
    print(f"{ts()} after  4, 5, 6")


asyncio.run(main())

0.001 before 1
0.002 Async action  started 1
2.004 Async action finished 1
2.004 after  1
2.105 before 2
2.105 Async action  started 2
4.105 Async action finished 2
4.105 after  2
4.105 before 4, 5, 6
4.106 Async action  started 4
4.106 Async action  started 5
4.106 Async action  started 6
6.108 Async action finished 4
6.108 Async action finished 5
6.109 Async action finished 6
6.109 after  4, 5, 6


In [1]:
# Wait the end of computation (async_action()) before get the next Item
# It's done introducing the queue in a custom operator (serialize_map_async())

import asyncio
from asyncio.futures import Future
import time
from collections import namedtuple
import rx
import rx.operators as ops
from rx.disposable import Disposable
from rx.subject import Subject
from rx.scheduler.eventloop import AsyncIOScheduler

import nest_asyncio
nest_asyncio.apply()

start = time.time()
def ts():
    return f"{time.time() - start:.3f}"

ACTION_DURATION = 2.0
Data = namedtuple('Data', ['id', 'future'])
subject = Subject()

async def async_action(id: int = 0):
    """ Some async processing, like sending/writing data. """
    print(f"{ts()} Async action  started {id}")
    await asyncio.sleep(ACTION_DURATION)
    print(f"{ts()} Async action finished {id}")
    return id


def serialize_map_async(mapper):
    def _serialize_map_async(source):
        def on_subscribe(observer, scheduler):
            q = asyncio.Queue()

            async def map_async(q):
                try:
                    while True:
                        i = await q.get()
                        ii = await mapper(i.id)
                        observer.on_next(ii)
                        i.future.set_result(ii)
                except Exception as e:
                    observer.on_error(e)

            def on_next(i):
                try:
                    q.put_nowait(i)
                except Exception as e:
                    observer.on_error(e)

            task = asyncio.create_task(map_async(q))
            d = source.subscribe(
                on_next=on_next,
                on_error=observer.on_error,
                on_completed=observer.on_completed,
            )

            def dispose():
                d.dispose()
                task.cancel()

            return Disposable(dispose)
        return rx.create(on_subscribe)
    return _serialize_map_async


async def setup():
    """ Setting up the Rx subject to make sure all work is done
        in sequence without overlaps.
    """

    subject.pipe(
        serialize_map_async(async_action),
    ).subscribe(
        scheduler=AsyncIOScheduler(asyncio.get_event_loop())
    )


async def add(id: int):
    """ Add "some work" and await until it is done. """
    future = asyncio.Future()
    subject.on_next(Data(id, future))
    return await future


async def main():
    await setup()

    # add work in any fashion (code I cannot influence)
    print(f"{ts()} before 1")
    await add(1)
    print(f"{ts()} after  1")

    await asyncio.sleep(0.1)

    print(f"{ts()} before 2")
    await add(2)
    print(f"{ts()} after  2")

    print(f"{ts()} before 4, 5, 6")
    await asyncio.gather(add(4), add(5), add(6))
    print(f"{ts()} after  4, 5, 6")


asyncio.run(main())

0.001 before 1
0.002 Async action  started 1
2.004 Async action finished 1
2.004 after  1
2.105 before 2
2.105 Async action  started 2
4.107 Async action finished 2
4.107 after  2
4.107 before 4, 5, 6
4.108 Async action  started 4
6.110 Async action finished 4
6.111 Async action  started 5
8.112 Async action finished 5
8.113 Async action  started 6
10.116 Async action finished 6
10.116 after  4, 5, 6


In [32]:
import enum

test = enum.Flag("Test",  [('CYAN', 4), ('MAGENTA', 5), ('YELLOW', 6)])
ok = test.CYAN | test.MAGENTA

In [36]:
test.CYAN in ok

True

In [34]:
test.YELLOW in ok

False

In [39]:
type(ok)

<enum 'Test'>

NameError: name 'python' is not defined

In [1]:
testSet = {'CYAN', 'MAGENTA', 'YELLOW'}
testSet

{'CYAN', 'MAGENTA', 'YELLOW'}

In [3]:
'CYANS' in testSet

False

In [4]:
testSet.add('RED')
testSet

{'CYAN', 'MAGENTA', 'RED', 'YELLOW'}

In [6]:
len({"ciao", "pao"} & {"1", "ciao", "3"})

1

In [16]:
def test(item):
   return item

def test_scan(acc, x):
    if acc != x:
        return x
    else:
        return 0

rx.from_([1,2,3,3,3,4,4,4,5,6,6,3,3]).pipe(
    ops.distinct_until_changed(lambda item: item),
    # ops.group_by(lambda item: item),
    # ops.flat_map(lambda obs: obs.pipe(ops.first())),
).subscribe(print, print, print)

1
2
3
4
5
6
3



In [63]:
class ShareMem:
    def __setattr__(self, key, value):
        print(f"shared_mem: {key} = {value}")
        return super().__setattr__(key, value)

    def __getattribute__(self, key):
        return super().__getattribute__(key)

    def setdefault(self, key, value):
        if not hasattr(self, key):
            setattr(self, key, value)
        return getattr(self, key)


ciao = ShareMem()
ciao.ok = 3

ciao.setdefault('ciao', 3)
ciao.setdefault('ok', 4)
print(ciao.ciao)
print(ciao.ok)


shared_mem: ok = 3
shared_mem: ciao = 3
3
3


In [1]:
""" IMPORT RXPY """
import rx
import rx.operators as ops
from rx.disposable import Disposable
from rx.subject import Subject
from rx.scheduler.eventloop import AsyncIOScheduler

from mape.utils import LogObserver

In [ ]:

virtual_room_temp = Subject().pipe(ops.start_with(False))

virtual_room_temp.pipe(ops.do_action(print)).subscribe()

In [26]:
import functools

def decorator_test(func):
    print("decorator")

    @functools.wraps(func)
    def wrapper_test(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_test

In [32]:
@decorator_test
def ciao(a, b):
    print(a, b)

# ciao = decorator_test(ciao)
ciao

decorator


<function __main__.ciao(a, b)>

In [34]:
ciao(1,2)

1 2


In [37]:
ciao = decorator_test(ciao)
ciao

decorator


<function __main__.ciao(a, b)>

In [38]:
ciao(1,2)

1 2


In [59]:
from functools import wraps, partial
import logging

def logged(func=None, *, level=logging.DEBUG, name=None, message=None):

    if func is None:
        print("with parentesi")
        return partial(logged, level=level, name=name, message=message)
    print("without parentesi")

    logname = name if name else func.__module__
    log = logging.getLogger(logname)
    logmsg = message if message else func.__name__
    @wraps(func)
    def wrapper(*args, **kwargs):
        #log.log(level, logmsg)
        return func(*args, **kwargs)
    return wrapper

# Example use
@logged
def add(x, y):
    return x + y

print("end")

@logged()
def sub(x, y):
    return x - y

print("end")

@logged(level=logging.CRITICAL, name='example')
def spam():
    print('Spam!')

print("end")

if __name__ == '__main__':
    import logging
    logging.basicConfig(level=logging.DEBUG)
    add(2,3)
    sub(2,3)
    spam()

without parentesi
end
with parentesi
without parentesi
end
with parentesi
without parentesi
end
Spam!


In [21]:
class MyDecorator(object):

    ok = "ok"

    """Decorator example mixing class and function definitions."""
    def __init__(self, func, param1, param2):
        self.func = func
        self.param1, self.param2 = param1, param2

    def __call__(self, *args, **kwargs):
        ...
        #use self.param1
        result = self.func(*args, **kwargs)
        #use self.param2
        return result

    def ciao(self):
        print("ciao")

def my_dec_factory(param1, param2):
    def decorator(func):
        ok = MyDecorator(func, param1, param2)
        ok.ciao = func
        return ok
    return decorator

In [22]:
@my_dec_factory(1, 2)
def say_whee():
    print("Whee!")

In [23]:
say_whee()

Whee!


In [24]:
say_whee.ciao()
say_whee.ok

Whee!


'ok'

In [1]:
[(i, j) for i in range(7) for j in range(10)]

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (3, 5),
 (3, 6),
 (3, 7),
 (3, 8),
 (3, 9),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4),
 (4, 5),
 (4, 6),
 (4, 7),
 (4, 8),
 (4, 9),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 4),
 (5, 5),
 (5, 6),
 (5, 7),
 (5, 8),
 (5, 9),
 (6, 0),
 (6, 1),
 (6, 2),
 (6, 3),
 (6, 4),
 (6, 5),
 (6, 6),
 (6, 7),
 (6, 8),
 (6, 9)]

In [12]:
!pip install ray rx rxray

  Using cached rxray-1.0.0.tar.gz (5.3 kB)
Using legacy 'setup.py install' for rxray, since package 'wheel' is not installed.
    Running setup.py install for rxray ... done
You should consider upgrading via the '/home/muttley/.pyenv/versions/3.8.12/envs/venv-3.8.12/bin/python3.8 -m pip install --upgrade pip' command.


In [25]:
import random

import rx
import rx.operators as ops
from rx.disposable import Disposable
from rx.subject import Subject
from rx.scheduler.eventloop import AsyncIOScheduler

import ray, rxray

In [28]:
data = [(i, j) for i in range(17) for j in range(1000)]
random.shuffle(data)
ray.init()

result = rx.from_(data).pipe(
    rxray.distribute(
        lambda: rx.pipe(
            ops.group_by(lambda i: i[0]),
            ops.flat_map(lambda g: g.pipe(
                ops.map(lambda i: i[1]),
                ops.average(),
                ops.map(lambda i: (g.key, i)),
            ))
        ),
        actor_selector=rxray.partition_by_key(lambda i: i[0]),
    ),
    ops.to_list(),
).run()

ray.shutdown()

# check that items are ordered by key
result

[(3, 499.5),
 (7, 499.5),
 (15, 499.5),
 (11, 499.5),
 (8, 499.5),
 (12, 499.5),
 (4, 499.5),
 (16, 499.5),
 (0, 499.5),
 (13, 499.5),
 (5, 499.5),
 (9, 499.5),
 (1, 499.5),
 (2, 499.5),
 (10, 499.5),
 (14, 499.5),
 (6, 499.5)]

In [59]:
ray.shutdown()

In [76]:
import time
import ray
import os
import numpy as np

from threading import current_thread

start = time.time()

def log(value: str):
    print(f"{time.time() - start:.3f} pid: {os.getpid()} {value}")

@ray.remote
def no_work(a):
    return

a_id = ray.put(np.zeros((100000, 2000)))
result_ids = [no_work.remote(a_id) for x in range(10)]
results = ray.get(result_ids)
print("duration =", time.time() - start)

ray.shutdown()

duration = 4.391823053359985


In [77]:
def ciao():
    return 1

dir(ciao)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [78]:
@ray.remote
def no_work(a):
    return


dir(no_work)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_accelerator_type',
 '_decorator',
 '_function',
 '_function_descriptor',
 '_function_name',
 '_function_signature',
 '_is_cross_language',
 '_language',
 '_last_export_session_and_job',
 '_max_calls',
 '_max_retries',
 '_memory',
 '_num_cpus',
 '_num_gpus',
 '_num_returns',
 '_object_store_memory',
 '_placement_group',
 '_remote',
 '_resources',
 '_retry_exceptions',
 '_runtime_env',
 '_uuid',
 'options',
 'remote']

In [17]:
def subscribe(*args, **kwargs):
    print(args)

    if 'port' in kwargs:
        print(kwargs['port'])
        del kwargs['port']
    print(kwargs)


subscribe("param", param2="param2", port="port")

('param',)
port
{'param2': 'param2'}


In [19]:
from rx.disposable import Disposable, CompositeDisposable

a = CompositeDisposable(Disposable(
    lambda: print("dispose1")),
    CompositeDisposable(
        Disposable(lambda: print("dispose2")),
                        Disposable(lambda: print("dispose3")))
)

a.dispose()


dispose1
dispose2
dispose3


In [46]:
from dataclasses import dataclass

@dataclass(frozen=True)
class PortClass:
    default: str
    p2p: str

test = PortClass("A", "B")

from typing import NamedTuple

PortTuple = NamedTuple("Point", [('x', int), ('y', int)])

PortTuple(x=1, y=2)

Point(x=1, y=2)

In [44]:
test.casa = "OK"

FrozenInstanceError: cannot assign to field 'casa'

In [34]:
ciao1.name

'casa'

In [1]:
import rx
import rx.operators as ops
from rx.disposable import Disposable, CompositeDisposable
from rx.subject import Subject

""" UNDERSTAND PIPE, OPERATORS, CALLING ORDER """

def fake_ops(tag):
    print(f"{tag} ops call 1")
    def _fake_ops(source):
        print(f"{tag} ops call 2")
        def subscribe(observer, scheduler = None):
            print(f"{tag} ops call 3 - subscribe")
            def on_next(value):
                print(f"{tag} ops call 4 - value throught")
                observer.on_next(value)

            return source.subscribe(
                on_next,
                observer.on_error,
                observer.on_completed,
                scheduler)
        return rx.create(subscribe)
    return _fake_ops

test = rx.of(1, 2, 3, 4, 5).pipe(fake_ops("A"), fake_ops("B"))
print("SUBSCRIBE")
dispose = test.subscribe(print)
dispose.dispose()

A ops call 1
B ops call 1
A ops call 2
B ops call 2
SUBSCRIBE
B ops call 3 - subscribe
A ops call 3 - subscribe
A ops call 4 - value throught
B ops call 4 - value throught
1
A ops call 4 - value throught
B ops call 4 - value throught
2
A ops call 4 - value throught
B ops call 4 - value throught
3
A ops call 4 - value throught
B ops call 4 - value throught
4
A ops call 4 - value throught
B ops call 4 - value throught
5


In [4]:
""" HOW DISPOSE WORKS """

def fake_obseravable():
    def on_subscribe(observer, scheduler):
        observer.on_next("A")
        observer.on_next("B")

        def dispose():
            print("fake_observable: DISPOSED")

        print("fake_observable")

        return Disposable(dispose)

    return rx.create(on_subscribe)

def lowercase():
    def _lowercase(source):
        def subscribe(observer, scheduler = None):
            def on_next(value):
                observer.on_next(value.lower())

            def dispose():
                print("lowercase operator: DISPOSED")

            print("lowercase")

            subscribe = source.subscribe(
                on_next,
                observer.on_error,
                observer.on_completed,
                scheduler)

            return CompositeDisposable(Disposable(dispose), subscribe)
        return rx.create(subscribe)
    return _lowercase

from mape.utils import LogObserver

print("subscribe")
a = fake_obseravable().pipe(lowercase()).subscribe(LogObserver("IO"))
print("dispose")
a.dispose()

subscribe
lowercase
(4662) on next: a | IO
(4662) on next: b | IO
fake_observable
dispose
lowercase operator: DISPOSED
fake_observable: DISPOSED


In [12]:
""" How ConnectableObservable/Multicast/publish NOT propagate dispose to Observable """

import mape
test = mape.BaseMapeElement2()

connectable_observable = fake_obseravable().pipe(lowercase(), ops.multicast(test))
print("subscribe")
a = test.subscribe(print)
print("connect")
dispose = connectable_observable.connect()
print("dispose subscribe")
a.dispose()
print("dispose connect")
dispose.dispose()


subscribe
connect
a
b
dispose subscribe
dispose connect
lowercase operator: DISPOSED
fake_observable: DISPOSED


In [17]:
ciao = Subject()

a = ciao.pipe(ops.publish())
sub = a.subscribe(print)
ciao.on_next(1)
b = a.connect()
ciao.on_next(2)
sub.dispose()
ciao.on_next(3)
b.dispose()
ciao.on_next(4)
a.subscribe(print)
ciao.on_next(5)
b = a.connect()
ciao.on_next(6)

2
6


In [49]:
import inspect

def test_func(*, ciao: str ="cia1o", bau=2):
    print(ciao, bau)
    pass

test_func()
test_func.__code__.co_code
test_func.__annotations__

for param in inspect.signature(test_func).parameters.values():
    print(param.name, param.default)

cia1o 2
ciao cia1o
bau 2


In [50]:
!pip install aioredis

     |████████████████████████████████| 71 kB 7.8 MB/s  eta 0:00:01
You should consider upgrading via the '/home/muttley/.pyenv/versions/3.8.12/envs/venv-3.8.12/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
import asyncio
import pickle
import mape

import nest_asyncio
nest_asyncio.apply()

import aioredis

class Bau:
    def __init__(self, boh) -> None:
        self.ok2 = True
        self.boh = boh

class Ciao:
    def __init__(self) -> None:
        super().__init__()
        self.ok = True
        self.bau = Bau(3)

async def main():
    redis = aioredis.from_url("redis://localhost")
    pickled_object = pickle.dumps(Ciao())
    await redis.set("my-key", pickled_object)
    unpacked_object = pickle.loads(await redis.get("my-key"))
    print(vars(unpacked_object.bau))


asyncio.run(main())

{'ok2': True, 'boh': 3}


In [17]:
import asyncio

import async_timeout

import aioredis

STOPWORD = "STOP"


async def reader(channel: aioredis.client.PubSub):
    while True:
        try:
            # async with async_timeout.timeout(1):
            message = await channel.get_message(ignore_subscribe_messages=True)
            if message is not None:
                print(f"(Reader) Message Received: {message}")
                if message["data"].decode() == STOPWORD:
                    print("(Reader) STOP")
                    break
            await asyncio.sleep(0.01)
        except asyncio.TimeoutError:
            pass


async def main():
    redis = aioredis.from_url("redis://localhost")
    pubsub = redis.pubsub()
    await pubsub.subscribe("channel:1", "channel:2")

    future = asyncio.create_task(reader(pubsub))

    await redis.publish("channel:1", "Hello")
    await asyncio.sleep(4)
    await redis.publish("channel:2", "World")
    await redis.publish("channel:1", STOPWORD)

    await future


if __name__ == "__main__":
    asyncio.run(main())

TypeError: 'Observable' object is not callable

In [3]:
!pip3 install pynput

(Reader) Message Received: {'type': 'message', 'pattern': None, 'channel': b'channel:1', 'data': b'Hello'}
(Reader) Message Received: {'type': 'message', 'pattern': None, 'channel': b'channel:2', 'data': b'World'}
(Reader) Message Received: {'type': 'message', 'pattern': None, 'channel': b'channel:1', 'data': b'STOP'}
(Reader) STOP


In [8]:
 """ Keyboard input in separate thread """

 from pynput.keyboard import Key, Listener

    def show(key):

        if key == Key.tab:
            print("good")

        if key != Key.tab:
            print("try again")

        # by pressing 'delete' button
        # you can terminate the loop
        if key == Key.delete:
            return False

    # Collect all event until released
    listener = Listener(on_press=show, suppress=True)
    listener.start()

     |████████████████████████████████| 89 kB 5.1 MB/s eta 0:00:011
     |████████████████████████████████| 179 kB 11.5 MB/s eta 0:00:01
Using legacy 'setup.py install' for evdev, since package 'wheel' is not installed.
    Running setup.py install for evdev ... done
You should consider upgrading via the '/home/muttley/.pyenv/versions/3.8.12/envs/venv-3.8.12/bin/python3.8 -m pip install --upgrade pip' command.


In [ ]:
def test(uno, due, tre=False, quattro=False):
    print(uno, due, tre, quattro)

import functools

new_test = functools.partial(test, *args, **kwarg)

new_test(True)

In [28]:

assert sdsdciao, "A title must be provided for OpenAPI, e.g.: 'My API'"
preint("ciao")

NameError: name 'args' is not defined

In [32]:
from enum import Flag
class Debug(Flag):
    DISABLE = 0
    IN = 1
    OUT = 2

ciao = Debug.OUT | Debug.IN

Debug.DISABLE in ciao

NameError: name 'sdsdciao' is not defined

In [43]:
class A:
    prefix = 'A'

class B(A):
    prefix = 'B'
    def __init__(self) -> None:
        self.prefix = 'sadsad'
        print(B.prefix, self.prefix, A.prefix)
        super().__init__()

class C(B):
    pass

c = C()
isinstance(c,Debug)

True

In [57]:
!pip install rxpy_backpressure

B sadsad A


False

In [44]:
!pip install prompt_toolkit

You should consider upgrading via the '/home/muttley/.pyenv/versions/3.8.12/envs/venv-3.8.12/bin/python3.8 -m pip install --upgrade pip' command.


In [8]:
from dataclasses import dataclass, field
import time
from typing import Type, Any, List, Tuple, Callable, Optional, Union, Awaitable, Coroutine, NamedTuple

from mape.utils import LogObserver

@dataclass
class Item:
    src: str = None
    dst: str = None
    hops: int = 0
    timestamp: float = field(default_factory=lambda: time.time())

@dataclass
class Message(Item):
    value: Any = None

@dataclass
class CallMethod(Item):
    name: str = None
    args: list = None
    kwargs: dict = None

items = []
items.append(Message(value=1, src='src0'))
# time.sleep(1)
items.append(Message(value=1, src='src0'))
items.append(Message(value=2, src='src0'))
items.append(CallMethod(name='method_name', args=['arg0'], kwargs={'kwarg0': 'kwarg0_value'}, src='src0'))
items.append(Message(value=1, src='src1'))
items.append(Message(value=2, src='src1'))
items.append(Message(value=3, src='src1'))


def _map(item):
    item.hops += 1
    return item


obs = rx.from_(items)

# Different pipe for each SRC
obs.pipe(
    ops.filter(lambda item: isinstance(item, Message)),
    ops.group_by(lambda item: item.src),
    ops.flat_map(
        lambda grp: grp.pipe(
            ops.distinct(lambda item: item.value),
            ops.average(lambda item: item.value),
            ops.map(lambda avg: Message(avg, src=grp.key))
        )
    )
).subscribe(LogObserver('group_by src'))

print()

obs.pipe(
    ops.filter(lambda item: isinstance(item, CallMethod)),
).subscribe(LogObserver('method call'))

print()

# Same pipe for all
obs.pipe(
    ops.filter(lambda item: isinstance(item, Message)),
    ops.map(_map),
    ops.distinct(lambda item: item.value),
).subscribe(LogObserver('global (not grouped)'))

print()

# Gateway draft
src2dest = { 'src0': LogObserver('dst0'), 'src1': LogObserver('dst1') }

def _send(item):
    src2dest[item.src].on_next(item)
 
obs.pipe(
    # ops.filter(lambda item: isinstance(item, Message)),
    ops.do_action(_send)

).subscribe()

on error: __init__() got multiple values for argument 'src' | group_by src

(7112) on next: CallMethod(src='src0', dst=None, hops=0, timestamp=1643559801.013751, name='method_name', args=['arg0'], kwargs={'kwarg0': 'kwarg0_value'}) | method call
completed | method call

(7043) on next: Message(src='src0', dst=None, hops=1, timestamp=1643559801.0135925, value=1) | global (not grouped)
(7043) on next: Message(src='src0', dst=None, hops=1, timestamp=1643559801.0136855, value=2) | global (not grouped)
(7043) on next: Message(src='src1', dst=None, hops=1, timestamp=1643559801.0138876, value=3) | global (not grouped)
completed | global (not grouped)

(1876) on next: Message(src='src0', dst=None, hops=1, timestamp=1643559801.0135925, value=1) | dst0
(1876) on next: Message(src='src0', dst=None, hops=1, timestamp=1643559801.0136423, value=1) | dst0
(1876) on next: Message(src='src0', dst=None, hops=1, timestamp=1643559801.0136855, value=2) | dst0
(1876) on next: CallMethod(src='src0', dst=None